## Calibration interrogation
Examine the parameter outputs from the calibration algorithm.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take approximately ten minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch tidy-notebooks
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import numpy as np
import arviz as az

from aust_covid.inputs import get_ifrs
from aust_covid.extra_text import add_calibration_blurb_to_tex
from aust_covid.calibration import get_priors
from emutools.tex import DummyTexDoc, StandardTexDoc, add_image_to_doc
from emutools.utils import load_param_info
from emutools.calibration import plot_param_progression, plot_posterior_comparison, tabulate_calib_results
from inputs.constants import SUPPLEMENT_PATH, RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN

In [ ]:
param_info = load_param_info()
parameters = param_info['value'].to_dict() | get_ifrs(DummyTexDoc())
priors = get_priors(PRIMARY_ANALYSIS in ['vacc', 'both'], param_info['abbreviations'], DummyTexDoc())
prior_names = [p.name for p in priors]
n_half_priors = round(len(priors) / 2)
first_priors = prior_names[:n_half_priors]
last_priors = prior_names[n_half_priors:]
first_third_priors = prior_names[:6]
middle_third_priors = prior_names[6: 12]
last_third_priors = prior_names[12:]

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

### Parameter traces with estimated posterior densities by chain

In [ ]:
trace_fig1 = plot_param_progression(idata, param_info['descriptions'], req_vars=first_third_priors)
trace_fig1

In [ ]:
trace_fig2 = plot_param_progression(idata, param_info['descriptions'], req_vars=middle_third_priors)
trace_fig2

In [ ]:
trace_fig3 = plot_param_progression(idata, param_info['descriptions'], req_vars=last_third_priors)
trace_fig3

### Comparison of parameter posteriors to priors

In [ ]:
comp_fig1 = plot_posterior_comparison(idata, priors, first_priors, param_info['abbreviations'].to_dict(), 0.995)
comp_fig1

In [ ]:
comp_fig2 = plot_posterior_comparison(idata, priors, last_priors, param_info['abbreviations'].to_dict(), 0.995)
comp_fig2

In [ ]:
calibration_table = tabulate_calib_results(idata, priors, param_info)
calibration_table

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'calibration', 'Calibration', 'austcovid')
app_doc.include_table(calibration_table, 'calibration_metrics', 'Calibration metrics', section='Calibration results', subsection='Calibration performance', col_splits=[0.142] * 7, table_width=12.0, longtable=True)
trace_figs = [trace_fig1, trace_fig2, trace_fig3]
comp_figs = [comp_fig1, comp_fig2]
for i_fig in range(1, 4):
    add_image_to_doc(trace_figs[i_fig - 1], f'trace_fig_{i_fig}', 'svg', f'Parameter posteriors and traces by chain, {i_fig}.', app_doc, 'Calibration results', subsection='Parameter inference')
for i_fig in range(1, 3):
    caption = 'Inferred parameter posterior densities (blue areas) compared against corresponding calibration algorithm prior distributions (grey areas).'
    add_image_to_doc(comp_figs[i_fig - 1], f'comp_fig_{i_fig}', 'svg', f'Posterior densities and prior distributions, {i_fig}.', app_doc, 'Calibration results', subsection='Parameter inference', caption=caption)
add_calibration_blurb_to_tex(app_doc)
app_doc.write_doc(order=['Calibration methods', 'Calibration results'])

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install inkscape texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex -shell-escape calibration
    ! pdflatex -shell-escape calibration
    ! pdflatex -shell-escape calibration